<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%94%D0%9E%D0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Динамические окрестностные модели (ДОМ)
####Вариант 5

In [15]:
import numpy as np
import pandas as pd
import random
import copy
import math
from prettytable import PrettyTable
import itertools as it

In [43]:
#A={a1, a2, a3}

#какие дуги входят в позицию:
#Ox[a1]={a1}
#Ox[a2]={}
#Ox[a3]={a1, a2}

#в какие позиции входят v
#Ov[a1]={a1}
#Ov[a2]={a2}
#Ov[a3]={a3}

#количества состояний/входов/выходов
elem = 100
count = [3, 3, 3]
Sx = np.array([[1, 0, 1], [0, 0, 1], [0, 0, 0]])
Sv = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

print("Матрица смежности:")
print("Sx = ")
print(Sx)
print("Sv = ")
print(Sv)

Матрица смежности:
Sx = 
[[1 0 1]
 [0 0 1]
 [0 0 0]]
Sv = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [17]:
def x1(x, v):
  return math.sin(math.log(abs(v[1] + v[0])) / x[2])

def x2(x, v):
  return math.cos(v[0] * x[2] - (x[1] / x[2])**3)

def x3(x, v):
  return math.sin(math.tan(v[0] / v[2]) - 3**(x[2]))

def v1(x, v):
  return (x[1] * v[0] * v[1] + math.tan(x[2])**2)

def v2(x, v):
  return (0.2 * v[0] + 33.0 * v[1] + v[2])

def v3(x, v):
  return (math.exp(x[0]) + .45 * math.log2(abs(x[1] * x[2])))

def y1(x, v):
  return (x[0] / v[0]) * math.sqrt(abs(x[0] / x[1]))

def y2(x, v):
  return math.sqrt(abs(math.sin(v[2])))

def y3(x, v):
  return v[2] * v[0] / v[0]

functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]

In [18]:
#Метод псевдообращения Бена-Израиля
def BenIsrael(A, eps = 0.000001): #А - np.array, eps - точность вычислительная
  I = np.diag(np.diag(np.ones([len(A),len(A)])))
  buf = 1
  i = 1

  if(np.linalg.norm(A, ord='fro') != 0): #если не нулевая норма
    gamma = 1.6 / (np.linalg.norm(A, ord='fro')**2)
    A1 = gamma * (A.T)
    i += 1
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
  else: #если нулевая норма
    A1 = np.zeros((len(A[0]), len(A)))
    buf = 0

  while(E >= eps and buf):
    A1 = A2.copy()
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
    i += 1

  return A1

#решение СЛАУ вида AX=B
def solvingSLAE(A, B):
  A = np.array(A.copy())

  #проверка на бесконечное количество решений
  A_psev = BenIsrael(A)

  X = A_psev.dot(B)
  return X

#=================================================================
#Генерация случайного вектора из 3-х элементов в диапазоне от -3 до 3
def generationRandomVector(elem = 3, start = -3, end = 3):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================
#создание столбца таблицы
def createVariants(function, x, v, typeFunc = 1, elem = 100):
  vector = []

  if(typeFunc == 0):
    for i in range(elem):
      vector.append(function(generationRandomVector(), generationRandomVector()))
  else:
    for i in range(elem):
      vector.append(function(x[i], v[i]))

  return vector

#=================================================================
#Создание ДОМ в определённый момент времени
def createTableDNM(functions, count, step, oldTable = []): #count(x, v, y)
  table = []
  vector = []
  columns = createColumns(step, count)

  if(step == 0):
    for i in range(sum(count[0:2])):
      table.append(createVariants(functions[i], [], [], 0)) #xi, vi

  else:
    for i in range(count[0]): #xi
      table.append(createVariants(functions[i], (np.array(oldTable))[:, 0:count[0]], (np.array(oldTable))[:, count[0]:sum(count[0:2])]))
    for i in range(count[1]): #vi
      table.append(createVariants(functions[count[0] + i], [], [], 0))

  for i in range(count[2]): #yi
    table.append(createVariants(functions[sum(count[0:2]) + i], (np.array(table).T)[:, 0:count[0]], (np.array(table).T)[:, count[0]:sum(count[0:2])]))

  table = pd.DataFrame(np.array(table).T, columns = columns)
  return table

#заполнение
def createColumns(step, count):
  columns = []
  for i in range(count[0]):
    columns.append("x" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[1]):
    columns.append("v" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[2]):
    columns.append("y" + str(i + 1) + "(" + str(step) + ")")
  return columns

In [19]:
#functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]
model_0 = createTableDNM(functions, count, 0)
model_0

,x1(0),x2(0),x3(0),v1(0),v2(0),v3(0),y1(0),y2(0),y3(0)
0,-0.819365,-0.995203,0.292472,4.611705,7.439431,5.077173,-0.161212,0.966541,5.077173
1,-0.555930,0.737661,0.661304,6.203309,74.809250,0.141772,-0.077800,0.375896,0.141772
2,0.313837,-0.016499,-0.174469,1210.240166,-37.560907,2.196077,0.001131,0.900444,2.196077
3,-0.247943,-0.554989,-0.963519,29.166551,84.028476,1.501099,-0.005682,0.998785,1.501099
4,0.566678,0.486031,0.190724,0.403516,-13.271804,2.261504,1.516392,0.877949,2.261504
...,...,...,...,...,...,...,...,...,...
95,-0.187149,-0.113713,-0.516601,4.373741,-18.623986,1.472776,-0.054894,0.997597,1.472776
96,0.255915,0.243548,0.541717,-1.116298,-9.410031,11.157229,-0.235002,0.993460,11.157229
97,-0.407042,-0.310138,0.998566,9.452555,-23.591916,2.012458,-0.049332,0.950811,2.012458
98,-0.273597,-0.264328,0.776110,0.245492,-85.455725,0.353551,-1.133855,0.588414,0.353551


In [20]:
model_1 = createTableDNM(functions, count, 1, np.array(model_0))
model_1

,x1(1),x2(1),x3(1),v1(1),v2(1),v3(1),y1(1),y2(1),y3(1)
0,0.791961,-0.995668,-0.096862,25.659124,94.742735,1.770056,0.027527,0.990057,1.770056
1,0.354312,-0.910109,-0.746602,3.796630,93.686842,1.499049,0.058228,0.998713,1.499049
2,-0.328533,-0.787584,0.172014,-5.149545,31.536383,3.946012,0.041205,0.848780,3.946012
3,0.980896,-0.999814,0.304135,-2.353457,12.205860,1.026503,-0.412828,0.924929,1.026503
4,0.737074,-0.721893,-0.868790,-9.777716,93.822572,0.762970,-0.076172,0.831307,0.762970
...,...,...,...,...,...,...,...,...,...
95,0.908795,-0.643718,-0.674653,7.618676,61.808298,2.063262,0.141733,0.938706,2.063262
96,-0.933346,0.767675,-0.941793,44.399942,99.388795,3.471880,-0.023179,0.569487,3.471880
97,0.469589,-0.999024,-0.697944,-1.785686,-31.071963,2.453629,-0.180295,0.796847,2.453629
98,-0.527579,0.973659,-0.998339,1.912318,57.262309,19.501446,-0.203080,0.778903,19.501446


In [21]:
model_2 = createTableDNM(functions, count, 2, np.array(model_1))
model_2

,x1(2),x2(2),x3(2),v1(2),v2(2),v3(2),y1(2),y2(2),y3(2)
0,0.721039,-0.040352,0.409427,-0.643457,95.143920,18.497840,-4.736819,0.586949,18.497840
1,0.148598,-0.066374,-0.907635,27.100659,78.783211,-0.428721,0.008204,0.644754,-0.428721
2,0.176230,0.659887,0.985773,11.842385,-81.561806,1.440412,0.007690,0.995747,1.440412
3,0.945408,-0.968033,-0.257961,3.433693,-67.621307,11.844519,0.272096,0.812881,11.844519
4,0.925589,-0.067068,-0.596595,10.150530,45.524950,1.026560,0.338751,0.924945,1.026560
...,...,...,...,...,...,...,...,...,...
95,-0.001936,0.962542,0.137421,-0.062867,51.445772,4.994432,0.001381,0.980045,4.994432
96,0.845647,-0.907590,-0.129187,306.353412,33.399828,0.843034,0.002665,0.864098,0.843034
97,0.957917,-0.115324,-0.976897,11.899012,10.229355,0.804499,0.232018,0.848813,0.804499
98,0.810901,0.555787,-0.233394,-14.736630,11.435242,2.580986,-0.066466,0.729178,2.580986


##1. Линейная форма функции пересчёта

###Для функции $x_1[t+1]$

In [44]:
#x3, v1, v2
vector_ones = [1] * elem
x1_linear = model_0[['x3(0)', 'v1(0)', 'v2(0)']]
x1_linear['x1(1)'] = model_1['x1(1)']
x1_linear.insert(0, "optional_col", vector_ones)
x1_linear

<ipython-input-44-08999791caa7>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1_linear['x1(1)'] = model_1['x1(1)']


,optional_col,x3(0),v1(0),v2(0),x1(1)
0,1,0.292472,4.611705,7.439431,0.791961
1,1,0.661304,6.203309,74.809250,0.354312
2,1,-0.174469,1210.240166,-37.560907,-0.328533
3,1,-0.963519,29.166551,84.028476,0.980896
4,1,0.190724,0.403516,-13.271804,0.737074
...,...,...,...,...,...
95,1,-0.516601,4.373741,-18.623986,0.908795
96,1,0.541717,-1.116298,-9.410031,-0.933346
97,1,0.998566,9.452555,-23.591916,0.469589
98,1,0.776110,0.245492,-85.455725,-0.527579


In [65]:
A1 = np.array(x1_linear)
#A1_psev = BenIsrael(A1[:, 0:(len(A1[0]) - 1)])
X1_estimation = solvingSLAE(A1[:, 0:(len(A1[0]) - 1)], np.matrix(A1[:, len(A1[0]) - 1]).T)
pd.DataFrame(X1_estimation)

,0
0,-0.061926
1,-0.264008
2,0.000029
3,-0.000781


###Для функции $x_2[t+1]$

In [23]:
#x2, x3, v1
x2_linear = model_0[['x2(0)', 'x3(0)', 'v1(0)']]
x2_linear['x2(1)'] = model_1['x2(1)']
x2_linear

<ipython-input-23-2fee96f88008>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x2_linear['x2(1)'] = model_1['x2(1)']


,x2(0),x3(0),v1(0),x2(1)
0,-0.995203,0.292472,4.611705,-0.995668
1,0.737661,0.661304,6.203309,-0.910109
2,-0.016499,-0.174469,1210.240166,-0.787584
3,-0.554989,-0.963519,29.166551,-0.999814
4,0.486031,0.190724,0.403516,-0.721893
...,...,...,...,...
95,-0.113713,-0.516601,4.373741,-0.643718
96,0.243548,0.541717,-1.116298,0.767675
97,-0.310138,0.998566,9.452555,-0.999024
98,-0.264328,0.776110,0.245492,0.973659


###Для функции $x_3[t+1]$

In [24]:
#x3, v1, v3
x3_linear = model_0[['x3(0)', 'v1(0)', 'v3(0)']]
x3_linear['x3(1)'] = model_1['x3(1)']
x3_linear

<ipython-input-24-b01b4a879c1c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x3_linear['x3(1)'] = model_1['x3(1)']


,x3(0),v1(0),v3(0),x3(1)
0,0.292472,4.611705,5.077173,-0.096862
1,0.661304,6.203309,0.141772,-0.746602
2,-0.174469,1210.240166,2.196077,0.172014
3,-0.963519,29.166551,1.501099,0.304135
4,0.190724,0.403516,2.261504,-0.868790
...,...,...,...,...
95,-0.516601,4.373741,1.472776,-0.674653
96,0.541717,-1.116298,11.157229,-0.941793
97,0.998566,9.452555,2.012458,-0.697944
98,0.776110,0.245492,0.353551,-0.998339


###Для функции $y_1[t]$

In [25]:
#x1, x2, v1
y1_linear = model_0[['x1(0)', 'x2(0)', 'v1(0)']]
y1_linear['y1(0)'] = model_0['y1(0)']
y1_linear

<ipython-input-25-a8e6ebb3674a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y1_linear['y1(0)'] = model_0['y1(0)']


,x1(0),x2(0),v1(0),y1(0)
0,-0.819365,-0.995203,4.611705,-0.161212
1,-0.555930,0.737661,6.203309,-0.077800
2,0.313837,-0.016499,1210.240166,0.001131
3,-0.247943,-0.554989,29.166551,-0.005682
4,0.566678,0.486031,0.403516,1.516392
...,...,...,...,...
95,-0.187149,-0.113713,4.373741,-0.054894
96,0.255915,0.243548,-1.116298,-0.235002
97,-0.407042,-0.310138,9.452555,-0.049332
98,-0.273597,-0.264328,0.245492,-1.133855


###Для функции $y_2[t]$

In [26]:
#v3
y2_linear = model_0[['v3(0)']]
y2_linear['y2(0)'] = model_0['y2(0)']
y2_linear

<ipython-input-26-256f5bc3816c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y2_linear['y2(0)'] = model_0['y2(0)']


,v3(0),y2(0)
0,5.077173,0.966541
1,0.141772,0.375896
2,2.196077,0.900444
3,1.501099,0.998785
4,2.261504,0.877949
...,...,...
95,1.472776,0.997597
96,11.157229,0.993460
97,2.012458,0.950811
98,0.353551,0.588414


###Для функции $y_3[t]$

In [27]:
#v1, v3
y3_linear = model_0[['v1(0)', 'v3(0)']]
y3_linear['y3(0)'] = model_0['y3(0)']
y3_linear

<ipython-input-27-1a8f0a0910b0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y3_linear['y3(0)'] = model_0['y3(0)']


,v1(0),v3(0),y3(0)
0,4.611705,5.077173,5.077173
1,6.203309,0.141772,0.141772
2,1210.240166,2.196077,2.196077
3,29.166551,1.501099,1.501099
4,0.403516,2.261504,2.261504
...,...,...,...
95,4.373741,1.472776,1.472776
96,-1.116298,11.157229,11.157229
97,9.452555,2.012458,2.012458
98,0.245492,0.353551,0.353551
